In [1]:
import numpy as np
import pandas as pd
import folium
from haversine import haversine

### 만들어 둔 CSV 파일 불러오기

  total_df : 밝기가 있는 주변 시설물  
  load_df : 도로 위의 좌표

In [12]:
total_df = pd.read_csv("C:\MyPython/1_data/total.csv", encoding = 'ms949')
total_df.drop('Unnamed: 0', axis=1, inplace=True)

load_df = pd.read_csv("2_2_total_load_points.csv", encoding = 'ms949')
load_df.drop('Unnamed: 0', axis=1, inplace=True)
load_df.reset_index(inplace =True,drop = True)
load_df.head(5)
#load_df   # 2036 rows × 8 columns

,SIG_CD,RDS_MAN_NO,RDS_MAN_NO2,RN,RN_CD,LAT,LNG,분류
0,11680,12,0,역삼로,3122008,37.493030,127.029968,도로
1,11680,12,1,역삼로,3122008,37.493107,127.030238,도로
2,11680,12,2,역삼로,3122008,37.493325,127.030953,도로
3,11680,12,3,역삼로,3122008,37.493571,127.031758,도로
4,11680,12,4,역삼로,3122008,37.493818,127.032570,도로
5,11680,12,5,역삼로,3122008,37.493880,127.032772,도로
6,11680,12,6,역삼로,3122008,37.493917,127.032895,도로
7,11680,12,7,역삼로,3122008,37.493961,127.033050,도로
8,11680,12,8,역삼로,3122008,37.494068,127.033398,도로
9,11680,12,9,역삼로,3122008,37.494083,127.033449,도로


### 중복데이터 제거

In [4]:
#a = total_df.groupby('명칭')['명칭'].count().sort_values(ascending = False)
#b = total_df.join(a, on='명칭', rsuffix='_r')
total_df = total_df.drop_duplicates(keep = 'first')
total_df.reset_index(drop=True, inplace = True)
print(len(total_df)) #1722
total_df.head(4)

1722


,분류,명칭,위도,경도
0,보안등,역삼1동833,37.493349,127.032033
1,보안등,역삼1동832-25,37.492834,127.031413
2,보안등,역삼1동832-27,37.492632,127.031508
3,보안등,역삼1동832-28,37.492480,127.031581


### 모든 데이터들을 담을 데이터 프레임을 만든다.

밝기 = 각 시설물의 점수      EX) 파출소 = 40점, 보안등 = 20점, 편의점 = 20점, cctv = 20점

                                 설문지 결과 : 파출소 > 보안등 > 편의점 > cctv
                             
총밝기 = '기준 시설물의 점수' + '주변 시설물의 점수'

In [10]:
all_df = pd.DataFrame(columns=['feature_ID', '밝기', '분류', '명칭', '위도', '경도'])

### 시설물에 밝기 점수 주기
for i in range(len(total_df)) :
    score = 0 
    if total_df['분류'][i] == '파출소' :
        score += 40
    elif total_df['분류'][i] == '보안등' :    
        score += 20
    elif total_df['분류'][i] == '편의점' :    
        score += 20
    else :    
        score += 20
        
    tmp = {'feature_ID' : 'Point_%d' % i, 
           '밝기' : score, 
           '분류' :total_df.loc[i,'분류'], '명칭' : total_df.loc[i,'명칭'], 
           '위도' : total_df.loc[i,'위도'], '경도' : total_df.loc[i,'경도'] 
           } 
    all_df = all_df.append(all_df.from_dict(tmp, orient = 'index').T)

all_df.reset_index(inplace=True,drop=True)
#all_df   # 1722 rows × 6 columns
all_df.head(3)

,feature_ID,밝기,분류,명칭,위도,경도
0,Point_0,20,보안등,역삼1동833,37.4933,127.032
1,Point_1,20,보안등,역삼1동832-25,37.4928,127.031
2,Point_2,20,보안등,역삼1동832-27,37.4926,127.032


In [11]:
# 시설물들
#load_df.loc[0,'LAT']

df = all_df
sp = [df.loc[0,'위도'],df.loc[0,'경도']] 

to_map1 = folium.Map(location= sp, zoom_start=15)

for n in range(0, 1000 ):
    folium.CircleMarker([df.loc[n,'위도'], df.loc[n,'경도']], popup= str(df.loc[n,'분류']), radius = int(df.loc[n,'밝기']/15)).add_to(to_map1)        

# 구분하기 쉽게 스타팅 지역은 빨간색 마커로 표시        
folium.Marker(sp, popup= 'SP', icon=folium.Icon(color='red')).add_to(to_map1)
#folium.Marker(ep, popup= 'EP', icon=folium.Icon(color='black')).add_to(to_map1)
to_map1


In [13]:
#엄청느림
# 총밝기 구하기  -> 교차점 정보에 주변 시설물들의 밝기 점수를 더해서 줌    
scores = []
for i in range(len(load_df)) :
    score = 0
    s_point = (load_df.loc[i,'LAT'], load_df.loc[i,'LNG']) # 시작점
    for n in range(len(all_df)) :
        t_point = (all_df.loc[n,'위도'], all_df.loc[n,'경도']) # 타겟
        d_m = haversine(s_point, t_point, unit='m') #  시작점과 타겟의 거리  단위 미터
        # 반경 30M 를 범위로 잡음
        if d_m <= 30 :
            score += all_df.loc[n, '밝기']
    # print(i, score) # 잘나옴
    scores.append(score)

In [14]:
# DataFrame 을 csv 파일로 저장하기
load_df['밝기'] = scores
load_df.to_csv("3_load_with_light.csv", mode='w', encoding='ms949')   # encoding 을 안 하면 한글이 깨짐

In [16]:
load_df

,SIG_CD,RDS_MAN_NO,RDS_MAN_NO2,RN,RN_CD,LAT,LNG,분류,밝기
0,11680,12,0,역삼로,3122008,37.493030,127.029968,도로,0
1,11680,12,1,역삼로,3122008,37.493107,127.030238,도로,0
2,11680,12,2,역삼로,3122008,37.493325,127.030953,도로,20
3,11680,12,3,역삼로,3122008,37.493571,127.031758,도로,0
4,11680,12,4,역삼로,3122008,37.493818,127.032570,도로,0
5,11680,12,5,역삼로,3122008,37.493880,127.032772,도로,20
6,11680,12,6,역삼로,3122008,37.493917,127.032895,도로,20
7,11680,12,7,역삼로,3122008,37.493961,127.033050,도로,0
8,11680,12,8,역삼로,3122008,37.494068,127.033398,도로,20
9,11680,12,9,역삼로,3122008,37.494083,127.033449,도로,20
